# Analisis Data Cuaca BMKG Indonesia Tahun 2019–2025

### Persiapan

## Menyiapkan library yang dibutuhkan

In [61]:
# Manipulasi Data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
%matplotlib inline
import seaborn as sns
import os
import glob


# Preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from imblearn.over_sampling import SMOTE, RandomOverSampler
from sklearn.model_selection import train_test_split

# Model
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB

# Evaluasi
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score

# Save Model
import pickle

import warnings
warnings.filterwarnings('ignore')

## Data wrangling

### Data Gathering 

In [67]:
# set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# Import dataset 2019
df_2019 = pd.read_excel('./data/data_2019/laporan_iklim_harian-250529135256.xlsx')
df_2019.head(50)

,Unnamed: 0,ID WMO,: 96783,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,NaN,NAMA STASIUN,: Stasiun Geofisika Bandung,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,LINTANG,: -6.88356,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,BUJUR,: 107.59733,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,ELEVASI,: 791 Meter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,TANGGAL,TN,TX,TAVG,RH_AVG,RR,SS,FF_X,DDD_X,FF_AVG,DDD_CAR
7,01-01-2019,21.8,28,23,79,0,1.7,3,280,2,W
8,02-01-2019,20.8,31,24,74,13.5,1.9,8,250,4,W
9,03-01-2019,20.7,30.6,24.1,76,0.8,7.2,7,260,4,W


##### Data Transformation Process

Untuk melakukan analisis data cuaca BMKG dengan baik, kita perlu melakukan beberapa transformasi data:

1. **Pembersihan Data**
    - Menghapus kolom yang tidak diperlukan untuk analisis
    - Menangani nilai-nilai khusus seperti 8888 (data tidak terukur) dan 9999 (tidak dilakukan pengukuran)
    - Menghilangkan baris keterangan dan informasi tambahan yang tidak relevan

2. **Konversi Format**
    - Mengubah data dari format Excel (.xlsx) ke CSV untuk mempercepat proses analisis
    - Memisahkan data berdasarkan bulan dan tahun untuk pengorganisasian yang lebih baik
    - Menggunakan struktur folder yang terstruktur (data/data_csv/tahun/bulan)

3. **Standarisasi Data**
    - Menyeragamkan nama kolom dan format data
    - Menambahkan label tanggal dalam format yang konsisten
    - Memastikan semua kolom memiliki nama dan tipe data yang tepat untuk memudahkan penggabungan data

Dengan transformasi ini, kita akan memiliki dataset yang bersih dan terstruktur untuk analisis lebih lanjut.

#### Transformasi dan Standarisasi Data Cuaca BMKG 2019–2025

In [ ]:
# merubah data 2019 ke csv dan menyimpannya ke folder data/data_csv/2019
os.makedirs('data/data_csv/2019', exist_ok=True)
column_names = ['Hari', 'Tn', 'Tx', 'Tavg', 'RH_avg', 'RR', 'ss', 'ff_x', 'ddd_x', 'ff_avg', 'ddd_car']

excel_files = sorted(glob.glob('./data/data_2019/*.xlsx'))

for file_path in excel_files:
    df = pd.read_excel(file_path, 
                       skiprows=8,
                       header=None,
                       names=column_names,
                       decimal=',',
                       engine='openpyxl')
    df = df.dropna(how='all')
    df = df[~df.iloc[:, 0].astype(str).str.contains(
        'KETERANGAN|Data tidak terukur|Tidak Ada Data|Tn:|Tx:|Tavg:|RH_avg:|RR:|ss:|ff_x:|ddd_x:|ff_avg:|ddd_car:', 
        na=False
    )]
    df = df.reset_index(drop=True)
    
    # Deteksi bulan dari data (asumsi kolom Hari berisi tanggal dalam format dd-mm-yyyy atau dd/mm/yyyy)
    bulan = None
    for val in df['Hari']:
        try:
            # Coba parse tanggal
            tgl = pd.to_datetime(str(val), dayfirst=True, errors='coerce')
            if pd.notnull(tgl):
                bulan = tgl.month
                break
        except:
            continue
    if bulan:
        nama_bulan = pd.to_datetime(f'2020-{bulan}-01').strftime('%b').lower()  # ex: jan, feb, mar
        output_path = f'data/data_csv/2019/data_bmkg_2019_{nama_bulan}.csv'
    else:
        # fallback jika tidak terdeteksi
        base_name = os.path.splitext(os.path.basename(file_path))[0]
        output_path = f'data/data_csv/2019/{base_name}.csv'
    
    df.to_csv(output_path, index=False)
    print(f"Data dari {file_path} berhasil disimpan ke {output_path}")

Data dari ./data/data_2019/laporan_iklim_harian-250529135256.xlsx berhasil disimpan ke data/data_csv/2019/data_bmkg_2019_jan.csv
Data dari ./data/data_2019/laporan_iklim_harian-250529135646.xlsx berhasil disimpan ke data/data_csv/2019/data_bmkg_2019_feb.csv
Data dari ./data/data_2019/laporan_iklim_harian-250529135702.xlsx berhasil disimpan ke data/data_csv/2019/data_bmkg_2019_mar.csv
Data dari ./data/data_2019/laporan_iklim_harian-250529135716.xlsx berhasil disimpan ke data/data_csv/2019/data_bmkg_2019_apr.csv
Data dari ./data/data_2019/laporan_iklim_harian-250529135733.xlsx berhasil disimpan ke data/data_csv/2019/data_bmkg_2019_may.csv
Data dari ./data/data_2019/laporan_iklim_harian-250529135749.xlsx berhasil disimpan ke data/data_csv/2019/data_bmkg_2019_jun.csv
Data dari ./data/data_2019/laporan_iklim_harian-250529135819.xlsx berhasil disimpan ke data/data_csv/2019/data_bmkg_2019_jul.csv
Data dari ./data/data_2019/laporan_iklim_harian-250529135841.xlsx berhasil disimpan ke data/data_

In [64]:
# merubah data 2020 ke csv dan menyimpannya ke folder data/data_csv/2020
os.makedirs('data/data_csv/2020', exist_ok=True)
column_names = ['Hari', 'Tn', 'Tx', 'Tavg', 'RH_avg', 'RR', 'ss', 'ff_x', 'ddd_x', 'ff_avg', 'ddd_car']

excel_files = sorted(glob.glob('./data/data_2020/*.xlsx'))

for file_path in excel_files:
    df = pd.read_excel(file_path, 
                       skiprows=8,
                       header=None,
                       names=column_names,
                       decimal=',',
                       engine='openpyxl')
    df = df.dropna(how='all')
    df = df[~df.iloc[:, 0].astype(str).str.contains(
        'KETERANGAN|Data tidak terukur|Tidak Ada Data|Tn:|Tx:|Tavg:|RH_avg:|RR:|ss:|ff_x:|ddd_x:|ff_avg:|ddd_car:', 
        na=False
    )]
    df = df.reset_index(drop=True)
    
    # Deteksi bulan dari data (asumsi kolom Hari berisi tanggal dalam format dd-mm-yyyy atau dd/mm/yyyy)
    bulan = None
    for val in df['Hari']:
        try:
            # Coba parse tanggal
            tgl = pd.to_datetime(str(val), dayfirst=True, errors='coerce')
            if pd.notnull(tgl):
                bulan = tgl.month
                break
        except:
            continue
    if bulan:
        nama_bulan = pd.to_datetime(f'2020-{bulan}-01').strftime('%b').lower()  # ex: jan, feb, mar
        output_path = f'data/data_csv/2020/data_bmkg_2020_{nama_bulan}.csv'
    else:
        # fallback jika tidak terdeteksi
        base_name = os.path.splitext(os.path.basename(file_path))[0]
        output_path = f'data/data_csv/2020/{base_name}.csv'
    
    df.to_csv(output_path, index=False)
    print(f"Data dari {file_path} berhasil disimpan ke {output_path}")

Data dari ./data/data_2020/laporan_iklim_harian-250529141528.xlsx berhasil disimpan ke data/data_csv/2020/data_bmkg_2020_jan.csv
Data dari ./data/data_2020/laporan_iklim_harian-250529141716.xlsx berhasil disimpan ke data/data_csv/2020/data_bmkg_2020_feb.csv
Data dari ./data/data_2020/laporan_iklim_harian-250529141735.xlsx berhasil disimpan ke data/data_csv/2020/data_bmkg_2020_mar.csv
Data dari ./data/data_2020/laporan_iklim_harian-250529141747.xlsx berhasil disimpan ke data/data_csv/2020/data_bmkg_2020_apr.csv
Data dari ./data/data_2020/laporan_iklim_harian-250529141757.xlsx berhasil disimpan ke data/data_csv/2020/data_bmkg_2020_may.csv
Data dari ./data/data_2020/laporan_iklim_harian-250529141807.xlsx berhasil disimpan ke data/data_csv/2020/data_bmkg_2020_jun.csv
Data dari ./data/data_2020/laporan_iklim_harian-250529141817.xlsx berhasil disimpan ke data/data_csv/2020/data_bmkg_2020_jul.csv
Data dari ./data/data_2020/laporan_iklim_harian-250529141829.xlsx berhasil disimpan ke data/data_

In [68]:
# merubah data 2021 ke csv dan menyimpannya ke folder data/data_csv/2021
os.makedirs('data/data_csv/2021', exist_ok=True)
column_names = ['Hari', 'Tn', 'Tx', 'Tavg', 'RH_avg', 'RR', 'ss', 'ff_x', 'ddd_x', 'ff_avg', 'ddd_car']

excel_files = sorted(glob.glob('./data/data_2021/*.xlsx'))

for file_path in excel_files:
    df = pd.read_excel(file_path, 
                       skiprows=8,
                       header=None,
                       names=column_names,
                       decimal=',',
                       engine='openpyxl')
    df = df.dropna(how='all')
    df = df[~df.iloc[:, 0].astype(str).str.contains(
        'KETERANGAN|Data tidak terukur|Tidak Ada Data|Tn:|Tx:|Tavg:|RH_avg:|RR:|ss:|ff_x:|ddd_x:|ff_avg:|ddd_car:', 
        na=False
    )]
    df = df.reset_index(drop=True)
    
    # Deteksi bulan dari data (asumsi kolom Hari berisi tanggal dalam format dd-mm-yyyy atau dd/mm/yyyy)
    bulan = None
    for val in df['Hari']:
        try:
            # Coba parse tanggal
            tgl = pd.to_datetime(str(val), dayfirst=True, errors='coerce')
            if pd.notnull(tgl):
                bulan = tgl.month
                break
        except:
            continue
    if bulan:
        nama_bulan = pd.to_datetime(f'2021-{bulan}-01').strftime('%b').lower()  # ex: jan, feb, mar
        output_path = f'data/data_csv/2021/data_bmkg_2021_{nama_bulan}.csv'
    else:
        # fallback jika tidak terdeteksi
        base_name = os.path.splitext(os.path.basename(file_path))[0]
        output_path = f'data/data_csv/2021/{base_name}.csv'
    
    df.to_csv(output_path, index=False)
    print(f"Data dari {file_path} berhasil disimpan ke {output_path}")

Data dari ./data/data_2021/laporan_iklim_harian-250529142706.xlsx berhasil disimpan ke data/data_csv/2021/data_bmkg_2021_jan.csv
Data dari ./data/data_2021/laporan_iklim_harian-250529142722.xlsx berhasil disimpan ke data/data_csv/2021/data_bmkg_2021_feb.csv
Data dari ./data/data_2021/laporan_iklim_harian-250529142733.xlsx berhasil disimpan ke data/data_csv/2021/data_bmkg_2021_mar.csv
Data dari ./data/data_2021/laporan_iklim_harian-250529142744.xlsx berhasil disimpan ke data/data_csv/2021/data_bmkg_2021_apr.csv
Data dari ./data/data_2021/laporan_iklim_harian-250529142754.xlsx berhasil disimpan ke data/data_csv/2021/data_bmkg_2021_may.csv
Data dari ./data/data_2021/laporan_iklim_harian-250529142803.xlsx berhasil disimpan ke data/data_csv/2021/data_bmkg_2021_jun.csv
Data dari ./data/data_2021/laporan_iklim_harian-250529142814.xlsx berhasil disimpan ke data/data_csv/2021/data_bmkg_2021_jul.csv
Data dari ./data/data_2021/laporan_iklim_harian-250529142824.xlsx berhasil disimpan ke data/data_

In [69]:
# merubah data 2021 ke csv dan menyimpannya ke folder data/data_csv/2021
os.makedirs('data/data_csv/2021', exist_ok=True)
column_names = ['Hari', 'Tn', 'Tx', 'Tavg', 'RH_avg', 'RR', 'ss', 'ff_x', 'ddd_x', 'ff_avg', 'ddd_car']

excel_files = sorted(glob.glob('./data/data_2021/*.xlsx'))

for file_path in excel_files:
    df = pd.read_excel(file_path, 
                       skiprows=8,
                       header=None,
                       names=column_names,
                       decimal=',',
                       engine='openpyxl')
    df = df.dropna(how='all')
    df = df[~df.iloc[:, 0].astype(str).str.contains(
        'KETERANGAN|Data tidak terukur|Tidak Ada Data|Tn:|Tx:|Tavg:|RH_avg:|RR:|ss:|ff_x:|ddd_x:|ff_avg:|ddd_car:', 
        na=False
    )]
    df = df.reset_index(drop=True)
    
    # Deteksi bulan dari data (asumsi kolom Hari berisi tanggal dalam format dd-mm-yyyy atau dd/mm/yyyy)
    bulan = None
    for val in df['Hari']:
        try:
            # Coba parse tanggal
            tgl = pd.to_datetime(str(val), dayfirst=True, errors='coerce')
            if pd.notnull(tgl):
                bulan = tgl.month
                break
        except:
            continue
    if bulan:
        nama_bulan = pd.to_datetime(f'2021-{bulan}-01').strftime('%b').lower()  # ex: jan, feb, mar
        output_path = f'data/data_csv/2021/data_bmkg_2021_{nama_bulan}.csv'
    else:
        # fallback jika tidak terdeteksi
        base_name = os.path.splitext(os.path.basename(file_path))[0]
        output_path = f'data/data_csv/2021/{base_name}.csv'
    
    df.to_csv(output_path, index=False)
    print(f"Data dari {file_path} berhasil disimpan ke {output_path}")

Data dari ./data/data_2021/laporan_iklim_harian-250529142706.xlsx berhasil disimpan ke data/data_csv/2021/data_bmkg_2021_jan.csv
Data dari ./data/data_2021/laporan_iklim_harian-250529142722.xlsx berhasil disimpan ke data/data_csv/2021/data_bmkg_2021_feb.csv
Data dari ./data/data_2021/laporan_iklim_harian-250529142733.xlsx berhasil disimpan ke data/data_csv/2021/data_bmkg_2021_mar.csv
Data dari ./data/data_2021/laporan_iklim_harian-250529142744.xlsx berhasil disimpan ke data/data_csv/2021/data_bmkg_2021_apr.csv
Data dari ./data/data_2021/laporan_iklim_harian-250529142754.xlsx berhasil disimpan ke data/data_csv/2021/data_bmkg_2021_may.csv
Data dari ./data/data_2021/laporan_iklim_harian-250529142803.xlsx berhasil disimpan ke data/data_csv/2021/data_bmkg_2021_jun.csv
Data dari ./data/data_2021/laporan_iklim_harian-250529142814.xlsx berhasil disimpan ke data/data_csv/2021/data_bmkg_2021_jul.csv
Data dari ./data/data_2021/laporan_iklim_harian-250529142824.xlsx berhasil disimpan ke data/data_

In [70]:
# merubah data 2022 ke csv dan menyimpannya ke folder data/data_csv/2022
os.makedirs('data/data_csv/2022', exist_ok=True)
column_names = ['Hari', 'Tn', 'Tx', 'Tavg', 'RH_avg', 'RR', 'ss', 'ff_x', 'ddd_x', 'ff_avg', 'ddd_car']

excel_files = sorted(glob.glob('./data/data_2022/*.xlsx'))

for file_path in excel_files:
    df = pd.read_excel(file_path, 
                       skiprows=8,
                       header=None,
                       names=column_names,
                       decimal=',',
                       engine='openpyxl')
    df = df.dropna(how='all')
    df = df[~df.iloc[:, 0].astype(str).str.contains(
        'KETERANGAN|Data tidak terukur|Tidak Ada Data|Tn:|Tx:|Tavg:|RH_avg:|RR:|ss:|ff_x:|ddd_x:|ff_avg:|ddd_car:', 
        na=False
    )]
    df = df.reset_index(drop=True)
    
    # Deteksi bulan dari data (asumsi kolom Hari berisi tanggal dalam format dd-mm-yyyy atau dd/mm/yyyy)
    bulan = None
    for val in df['Hari']:
        try:
            # Coba parse tanggal
            tgl = pd.to_datetime(str(val), dayfirst=True, errors='coerce')
            if pd.notnull(tgl):
                bulan = tgl.month
                break
        except:
            continue
    if bulan:
        nama_bulan = pd.to_datetime(f'2022-{bulan}-01').strftime('%b').lower()  # ex: jan, feb, mar
        output_path = f'data/data_csv/2022/data_bmkg_2022_{nama_bulan}.csv'
    else:
        # fallback jika tidak terdeteksi
        base_name = os.path.splitext(os.path.basename(file_path))[0]
        output_path = f'data/data_csv/2022/{base_name}.csv'
    
    df.to_csv(output_path, index=False)
    print(f"Data dari {file_path} berhasil disimpan ke {output_path}")

Data dari ./data/data_2022/laporan_iklim_harian-250529143258.xlsx berhasil disimpan ke data/data_csv/2022/data_bmkg_2022_jan.csv
Data dari ./data/data_2022/laporan_iklim_harian-250529143312.xlsx berhasil disimpan ke data/data_csv/2022/data_bmkg_2022_jan.csv
Data dari ./data/data_2022/laporan_iklim_harian-250529143325.xlsx berhasil disimpan ke data/data_csv/2022/data_bmkg_2022_mar.csv
Data dari ./data/data_2022/laporan_iklim_harian-250529143347.xlsx berhasil disimpan ke data/data_csv/2022/data_bmkg_2022_apr.csv
Data dari ./data/data_2022/laporan_iklim_harian-250529143404.xlsx berhasil disimpan ke data/data_csv/2022/data_bmkg_2022_may.csv
Data dari ./data/data_2022/laporan_iklim_harian-250529143416.xlsx berhasil disimpan ke data/data_csv/2022/data_bmkg_2022_jun.csv
Data dari ./data/data_2022/laporan_iklim_harian-250529143426.xlsx berhasil disimpan ke data/data_csv/2022/data_bmkg_2022_oct.csv
Data dari ./data/data_2022/laporan_iklim_harian-250529143440.xlsx berhasil disimpan ke data/data_

In [71]:
# merubah data 2023 ke csv dan menyimpannya ke folder data/data_csv/2023
os.makedirs('data/data_csv/2023', exist_ok=True)
column_names = ['Hari', 'Tn', 'Tx', 'Tavg', 'RH_avg', 'RR', 'ss', 'ff_x', 'ddd_x', 'ff_avg', 'ddd_car']

excel_files = sorted(glob.glob('./data/data_2023/*.xlsx'))

for file_path in excel_files:
    df = pd.read_excel(file_path, 
                       skiprows=8,
                       header=None,
                       names=column_names,
                       decimal=',',
                       engine='openpyxl')
    df = df.dropna(how='all')
    df = df[~df.iloc[:, 0].astype(str).str.contains(
        'KETERANGAN|Data tidak terukur|Tidak Ada Data|Tn:|Tx:|Tavg:|RH_avg:|RR:|ss:|ff_x:|ddd_x:|ff_avg:|ddd_car:', 
        na=False
    )]
    df = df.reset_index(drop=True)
    
    # Deteksi bulan dari data (asumsi kolom Hari berisi tanggal dalam format dd-mm-yyyy atau dd/mm/yyyy)
    bulan = None
    for val in df['Hari']:
        try:
            # Coba parse tanggal
            tgl = pd.to_datetime(str(val), dayfirst=True, errors='coerce')
            if pd.notnull(tgl):
                bulan = tgl.month
                break
        except:
            continue
    if bulan:
        nama_bulan = pd.to_datetime(f'2023-{bulan}-01').strftime('%b').lower()  # ex: jan, feb, mar
        output_path = f'data/data_csv/2023/data_bmkg_2023_{nama_bulan}.csv'
    else:
        # fallback jika tidak terdeteksi
        base_name = os.path.splitext(os.path.basename(file_path))[0]
        output_path = f'data/data_csv/2023/{base_name}.csv'
    
    df.to_csv(output_path, index=False)
    print(f"Data dari {file_path} berhasil disimpan ke {output_path}")

Data dari ./data/data_2023/laporan_iklim_harian-250529144012.xlsx berhasil disimpan ke data/data_csv/2023/data_bmkg_2023_nov.csv
Data dari ./data/data_2023/laporan_iklim_harian-250529144040.xlsx berhasil disimpan ke data/data_csv/2023/data_bmkg_2023_feb.csv
Data dari ./data/data_2023/laporan_iklim_harian-250529144049.xlsx berhasil disimpan ke data/data_csv/2023/data_bmkg_2023_mar.csv
Data dari ./data/data_2023/laporan_iklim_harian-250529144058.xlsx berhasil disimpan ke data/data_csv/2023/data_bmkg_2023_apr.csv
Data dari ./data/data_2023/laporan_iklim_harian-250529144111.xlsx berhasil disimpan ke data/data_csv/2023/data_bmkg_2023_may.csv
Data dari ./data/data_2023/laporan_iklim_harian-250529144119.xlsx berhasil disimpan ke data/data_csv/2023/data_bmkg_2023_jun.csv
Data dari ./data/data_2023/laporan_iklim_harian-250529144131.xlsx berhasil disimpan ke data/data_csv/2023/data_bmkg_2023_jul.csv
Data dari ./data/data_2023/laporan_iklim_harian-250529144140.xlsx berhasil disimpan ke data/data_

In [72]:
# merubah data 2024 ke csv dan menyimpannya ke folder data/data_csv/2024
os.makedirs('data/data_csv/2024', exist_ok=True)
column_names = ['Hari', 'Tn', 'Tx', 'Tavg', 'RH_avg', 'RR', 'ss', 'ff_x', 'ddd_x', 'ff_avg', 'ddd_car']

excel_files = sorted(glob.glob('./data/data_2024/*.xlsx'))

for file_path in excel_files:
    df = pd.read_excel(file_path, 
                       skiprows=8,
                       header=None,
                       names=column_names,
                       decimal=',',
                       engine='openpyxl')
    df = df.dropna(how='all')
    df = df[~df.iloc[:, 0].astype(str).str.contains(
        'KETERANGAN|Data tidak terukur|Tidak Ada Data|Tn:|Tx:|Tavg:|RH_avg:|RR:|ss:|ff_x:|ddd_x:|ff_avg:|ddd_car:', 
        na=False
    )]
    df = df.reset_index(drop=True)
    
    # Deteksi bulan dari data (asumsi kolom Hari berisi tanggal dalam format dd-mm-yyyy atau dd/mm/yyyy)
    bulan = None
    for val in df['Hari']:
        try:
            # Coba parse tanggal
            tgl = pd.to_datetime(str(val), dayfirst=True, errors='coerce')
            if pd.notnull(tgl):
                bulan = tgl.month
                break
        except:
            continue
    if bulan:
        nama_bulan = pd.to_datetime(f'2024-{bulan}-01').strftime('%b').lower()  # ex: jan, feb, mar
        output_path = f'data/data_csv/2024/data_bmkg_2024_{nama_bulan}.csv'
    else:
        # fallback jika tidak terdeteksi
        base_name = os.path.splitext(os.path.basename(file_path))[0]
        output_path = f'data/data_csv/2024/{base_name}.csv'
    
    df.to_csv(output_path, index=False)
    print(f"Data dari {file_path} berhasil disimpan ke {output_path}")

Data dari ./data/data_2024/laporan_iklim_harian-250529144535.xlsx berhasil disimpan ke data/data_csv/2024/data_bmkg_2024_jan.csv
Data dari ./data/data_2024/laporan_iklim_harian-250529144602.xlsx berhasil disimpan ke data/data_csv/2024/data_bmkg_2024_feb.csv
Data dari ./data/data_2024/laporan_iklim_harian-250529144609.xlsx berhasil disimpan ke data/data_csv/2024/data_bmkg_2024_mar.csv
Data dari ./data/data_2024/laporan_iklim_harian-250529144618.xlsx berhasil disimpan ke data/data_csv/2024/data_bmkg_2024_apr.csv
Data dari ./data/data_2024/laporan_iklim_harian-250529144627.xlsx berhasil disimpan ke data/data_csv/2024/data_bmkg_2024_may.csv
Data dari ./data/data_2024/laporan_iklim_harian-250529144641.xlsx berhasil disimpan ke data/data_csv/2024/data_bmkg_2024_jun.csv
Data dari ./data/data_2024/laporan_iklim_harian-250529144653.xlsx berhasil disimpan ke data/data_csv/2024/data_bmkg_2024_jul.csv
Data dari ./data/data_2024/laporan_iklim_harian-250529144704.xlsx berhasil disimpan ke data/data_

In [73]:
# merubah data 2025 ke csv dan menyimpannya ke folder data/data_csv/2025
os.makedirs('data/data_csv/2025', exist_ok=True)
column_names = ['Hari', 'Tn', 'Tx', 'Tavg', 'RH_avg', 'RR', 'ss', 'ff_x', 'ddd_x', 'ff_avg', 'ddd_car']

excel_files = sorted(glob.glob('./data/data_2025/*.xlsx'))

for file_path in excel_files:
    df = pd.read_excel(file_path, 
                       skiprows=8,
                       header=None,
                       names=column_names,
                       decimal=',',
                       engine='openpyxl')
    df = df.dropna(how='all')
    df = df[~df.iloc[:, 0].astype(str).str.contains(
        'KETERANGAN|Data tidak terukur|Tidak Ada Data|Tn:|Tx:|Tavg:|RH_avg:|RR:|ss:|ff_x:|ddd_x:|ff_avg:|ddd_car:', 
        na=False
    )]
    df = df.reset_index(drop=True)
    
    # Deteksi bulan dari data (asumsi kolom Hari berisi tanggal dalam format dd-mm-yyyy atau dd/mm/yyyy)
    bulan = None
    for val in df['Hari']:
        try:
            # Coba parse tanggal
            tgl = pd.to_datetime(str(val), dayfirst=True, errors='coerce')
            if pd.notnull(tgl):
                bulan = tgl.month
                break
        except:
            continue
    if bulan:
        nama_bulan = pd.to_datetime(f'2025-{bulan}-01').strftime('%b').lower()  # ex: jan, feb, mar
        output_path = f'data/data_csv/2025/data_bmkg_2025_{nama_bulan}.csv'
    else:
        # fallback jika tidak terdeteksi
        base_name = os.path.splitext(os.path.basename(file_path))[0]
        output_path = f'data/data_csv/2025/{base_name}.csv'
    
    df.to_csv(output_path, index=False)
    print(f"Data dari {file_path} berhasil disimpan ke {output_path}")

Data dari ./data/data_2025/laporan_iklim_harian-250529144942.xlsx berhasil disimpan ke data/data_csv/2025/data_bmkg_2025_jan.csv
Data dari ./data/data_2025/laporan_iklim_harian-250529144954.xlsx berhasil disimpan ke data/data_csv/2025/data_bmkg_2025_feb.csv
Data dari ./data/data_2025/laporan_iklim_harian-250529145003.xlsx berhasil disimpan ke data/data_csv/2025/data_bmkg_2025_mar.csv
Data dari ./data/data_2025/laporan_iklim_harian-250529145014.xlsx berhasil disimpan ke data/data_csv/2025/data_bmkg_2025_apr.csv
Data dari ./data/data_2025/laporan_iklim_harian-250529145027.xlsx berhasil disimpan ke data/data_csv/2025/data_bmkg_2025_may.csv


#### Data Merge

In [ ]:
# mengabungkan semua file csv berlabel bulan dari 2019 menjadi tahunan
os.makedirs('data/data_tahunan', exist_ok=True)

# Urutan bulan
bulan_order = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']

# Ambil semua file csv berlabel bulan
csv_files = glob.glob('data/2019/data_bmkg_2019_*.csv')

# Pasangkan file dengan urutan bulannya
csv_bulan = []
for f in csv_files:
    nama_bulan = os.path.basename(f).split('_')[-1].replace('.csv', '')
    if nama_bulan in bulan_order:
        csv_bulan.append((f, bulan_order.index(nama_bulan)))
    else:
        csv_bulan.append((f, 99))  # file yang tidak jelas bulannya di akhir

# Urutkan file berdasarkan bulan
csv_bulan_sorted = sorted(csv_bulan, key=lambda x: x[1])

# Gabungkan semua file sesuai urutan bulan
df_all = pd.concat([pd.read_csv(f[0]) for f in csv_bulan_sorted], ignore_index=True)

# Simpan hasil merge
output_path = 'data/data_tahunan/data_bmkg_2019_all.csv'
df_all.to_csv(output_path, index=False)
print(f"Data berhasil digabung dan disusun Jan-Des ke {output_path}")

Data berhasil digabung dan disusun Jan-Des ke data/data_tahunan/data_bmkg_2019_all.csv


In [78]:
# mengabungkan semua file csv berlabel bulan dari 2019 menjadi tahunan
os.makedirs('data/data_tahunan', exist_ok=True)

# Urutan bulan
bulan_order = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']

# Ambil semua file csv berlabel bulan
csv_files = glob.glob('data/2019/data_bmkg_2019_*.csv')

# Pasangkan file dengan urutan bulannya
csv_bulan = []
for f in csv_files:
    nama_bulan = os.path.basename(f).split('_')[-1].replace('.csv', '')
    if nama_bulan in bulan_order:
        csv_bulan.append((f, bulan_order.index(nama_bulan)))
    else:
        csv_bulan.append((f, 99))  # file yang tidak jelas bulannya di akhir

# Urutkan file berdasarkan bulan
csv_bulan_sorted = sorted(csv_bulan, key=lambda x: x[1])

# Gabungkan semua file sesuai urutan bulan
df_all = pd.concat([pd.read_csv(f[0]) for f in csv_bulan_sorted], ignore_index=True)

# Simpan hasil merge
output_path = 'data/data_tahunan/data_bmkg_2019_all.csv'
df_all.to_csv(output_path, index=False)
print(f"Data berhasil digabung dan disusun Jan-Des ke {output_path}")

Data berhasil digabung dan disusun Jan-Des ke data/data_tahunan/data_bmkg_2019_all.csv


In [79]:
# mengabungkan semua file csv berlabel bulan dari 2020 menjadi tahunan
os.makedirs('data/data_tahunan', exist_ok=True)

# Urutan bulan
bulan_order = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']

# Ambil semua file csv berlabel bulan
csv_files = glob.glob('data/2020/data_bmkg_2020_*.csv')

# Pasangkan file dengan urutan bulannya
csv_bulan = []
for f in csv_files:
    nama_bulan = os.path.basename(f).split('_')[-1].replace('.csv', '')
    if nama_bulan in bulan_order:
        csv_bulan.append((f, bulan_order.index(nama_bulan)))
    else:
        csv_bulan.append((f, 99))  # file yang tidak jelas bulannya di akhir

# Urutkan file berdasarkan bulan
csv_bulan_sorted = sorted(csv_bulan, key=lambda x: x[1])

# Gabungkan semua file sesuai urutan bulan
df_all = pd.concat([pd.read_csv(f[0]) for f in csv_bulan_sorted], ignore_index=True)

# Simpan hasil merge
output_path = 'data/data_tahunan/data_bmkg_2020_all.csv'
df_all.to_csv(output_path, index=False)
print(f"Data berhasil digabung dan disusun Jan-Des ke {output_path}")

Data berhasil digabung dan disusun Jan-Des ke data/data_tahunan/data_bmkg_2020_all.csv


In [80]:
# mengabungkan semua file csv berlabel bulan dari 2021 menjadi tahunan
os.makedirs('data/data_tahunan', exist_ok=True)

# Urutan bulan
bulan_order = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']

# Ambil semua file csv berlabel bulan
csv_files = glob.glob('data/2021/data_bmkg_2021_*.csv')

# Pasangkan file dengan urutan bulannya
csv_bulan = []
for f in csv_files:
    nama_bulan = os.path.basename(f).split('_')[-1].replace('.csv', '')
    if nama_bulan in bulan_order:
        csv_bulan.append((f, bulan_order.index(nama_bulan)))
    else:
        csv_bulan.append((f, 99))  # file yang tidak jelas bulannya di akhir

# Urutkan file berdasarkan bulan
csv_bulan_sorted = sorted(csv_bulan, key=lambda x: x[1])

# Gabungkan semua file sesuai urutan bulan
df_all = pd.concat([pd.read_csv(f[0]) for f in csv_bulan_sorted], ignore_index=True)

# Simpan hasil merge
output_path = 'data/data_tahunan/data_bmkg_2021_all.csv'
df_all.to_csv(output_path, index=False)
print(f"Data berhasil digabung dan disusun Jan-Des ke {output_path}")

Data berhasil digabung dan disusun Jan-Des ke data/data_tahunan/data_bmkg_2021_all.csv


In [ ]:
# mengabungkan semua file csv berlabel bulan dari 2020 menjadi tahunan
os.makedirs('data/data_tahunan', exist_ok=True)

# Urutan bulan
bulan_order = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']

# Ambil semua file csv berlabel bulan
csv_files = glob.glob('data/2020/data_bmkg_2020_*.csv')

# Pasangkan file dengan urutan bulannya
csv_bulan = []
for f in csv_files:
    nama_bulan = os.path.basename(f).split('_')[-1].replace('.csv', '')
    if nama_bulan in bulan_order:
        csv_bulan.append((f, bulan_order.index(nama_bulan)))
    else:
        csv_bulan.append((f, 99))  # file yang tidak jelas bulannya di akhir

# Urutkan file berdasarkan bulan
csv_bulan_sorted = sorted(csv_bulan, key=lambda x: x[1])

# Gabungkan semua file sesuai urutan bulan
df_all = pd.concat([pd.read_csv(f[0]) for f in csv_bulan_sorted], ignore_index=True)

# Simpan hasil merge
output_path = 'data/data_tahunan/data_bmkg_2020_all.csv'
df_all.to_csv(output_path, index=False)
print(f"Data berhasil digabung dan disusun Jan-Des ke {output_path}")

In [81]:
# mengabungkan semua file csv berlabel bulan dari 2022 menjadi tahunan
os.makedirs('data/data_tahunan', exist_ok=True)

# Urutan bulan
bulan_order = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']

# Ambil semua file csv berlabel bulan
csv_files = glob.glob('data/2022/data_bmkg_2022_*.csv')

# Pasangkan file dengan urutan bulannya
csv_bulan = []
for f in csv_files:
    nama_bulan = os.path.basename(f).split('_')[-1].replace('.csv', '')
    if nama_bulan in bulan_order:
        csv_bulan.append((f, bulan_order.index(nama_bulan)))
    else:
        csv_bulan.append((f, 99))  # file yang tidak jelas bulannya di akhir

# Urutkan file berdasarkan bulan
csv_bulan_sorted = sorted(csv_bulan, key=lambda x: x[1])

# Gabungkan semua file sesuai urutan bulan
df_all = pd.concat([pd.read_csv(f[0]) for f in csv_bulan_sorted], ignore_index=True)

# Simpan hasil merge
output_path = 'data/data_tahunan/data_bmkg_2022_all.csv'
df_all.to_csv(output_path, index=False)
print(f"Data berhasil digabung dan disusun Jan-Des ke {output_path}")

Data berhasil digabung dan disusun Jan-Des ke data/data_tahunan/data_bmkg_2022_all.csv


In [ ]:
# mengabungkan semua file csv berlabel bulan dari 2020 menjadi tahunan
os.makedirs('data/data_tahunan', exist_ok=True)

# Urutan bulan
bulan_order = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']

# Ambil semua file csv berlabel bulan
csv_files = glob.glob('data/2020/data_bmkg_2020_*.csv')

# Pasangkan file dengan urutan bulannya
csv_bulan = []
for f in csv_files:
    nama_bulan = os.path.basename(f).split('_')[-1].replace('.csv', '')
    if nama_bulan in bulan_order:
        csv_bulan.append((f, bulan_order.index(nama_bulan)))
    else:
        csv_bulan.append((f, 99))  # file yang tidak jelas bulannya di akhir

# Urutkan file berdasarkan bulan
csv_bulan_sorted = sorted(csv_bulan, key=lambda x: x[1])

# Gabungkan semua file sesuai urutan bulan
df_all = pd.concat([pd.read_csv(f[0]) for f in csv_bulan_sorted], ignore_index=True)

# Simpan hasil merge
output_path = 'data/data_tahunan/data_bmkg_2020_all.csv'
df_all.to_csv(output_path, index=False)
print(f"Data berhasil digabung dan disusun Jan-Des ke {output_path}")

In [82]:
# mengabungkan semua file csv berlabel bulan dari 2023 menjadi tahunan
os.makedirs('data/data_tahunan', exist_ok=True)

# Urutan bulan
bulan_order = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']

# Ambil semua file csv berlabel bulan
csv_files = glob.glob('data/2023/data_bmkg_2023_*.csv')

# Pasangkan file dengan urutan bulannya
csv_bulan = []
for f in csv_files:
    nama_bulan = os.path.basename(f).split('_')[-1].replace('.csv', '')
    if nama_bulan in bulan_order:
        csv_bulan.append((f, bulan_order.index(nama_bulan)))
    else:
        csv_bulan.append((f, 99))  # file yang tidak jelas bulannya di akhir

# Urutkan file berdasarkan bulan
csv_bulan_sorted = sorted(csv_bulan, key=lambda x: x[1])

# Gabungkan semua file sesuai urutan bulan
df_all = pd.concat([pd.read_csv(f[0]) for f in csv_bulan_sorted], ignore_index=True)

# Simpan hasil merge
output_path = 'data/data_tahunan/data_bmkg_2023_all.csv'
df_all.to_csv(output_path, index=False)
print(f"Data berhasil digabung dan disusun Jan-Des ke {output_path}")

Data berhasil digabung dan disusun Jan-Des ke data/data_tahunan/data_bmkg_2023_all.csv


In [83]:
# mengabungkan semua file csv berlabel bulan dari 2024 menjadi tahunan
os.makedirs('data/data_tahunan', exist_ok=True)

# Urutan bulan
bulan_order = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']

# Ambil semua file csv berlabel bulan
csv_files = glob.glob('data/2024/data_bmkg_2024_*.csv')

# Pasangkan file dengan urutan bulannya
csv_bulan = []
for f in csv_files:
    nama_bulan = os.path.basename(f).split('_')[-1].replace('.csv', '')
    if nama_bulan in bulan_order:
        csv_bulan.append((f, bulan_order.index(nama_bulan)))
    else:
        csv_bulan.append((f, 99))  # file yang tidak jelas bulannya di akhir

# Urutkan file berdasarkan bulan
csv_bulan_sorted = sorted(csv_bulan, key=lambda x: x[1])

# Gabungkan semua file sesuai urutan bulan
df_all = pd.concat([pd.read_csv(f[0]) for f in csv_bulan_sorted], ignore_index=True)

# Simpan hasil merge
output_path = 'data/data_tahunan/data_bmkg_2024_all.csv'
df_all.to_csv(output_path, index=False)
print(f"Data berhasil digabung dan disusun Jan-Des ke {output_path}")

Data berhasil digabung dan disusun Jan-Des ke data/data_tahunan/data_bmkg_2024_all.csv


In [84]:
# mengabungkan semua file csv berlabel bulan dari 2025 menjadi tahunan
os.makedirs('data/data_tahunan', exist_ok=True)

# Urutan bulan
bulan_order = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']

# Ambil semua file csv berlabel bulan
csv_files = glob.glob('data/2025/data_bmkg_2025_*.csv')

# Pasangkan file dengan urutan bulannya
csv_bulan = []
for f in csv_files:
    nama_bulan = os.path.basename(f).split('_')[-1].replace('.csv', '')
    if nama_bulan in bulan_order:
        csv_bulan.append((f, bulan_order.index(nama_bulan)))
    else:
        csv_bulan.append((f, 99))  # file yang tidak jelas bulannya di akhir

# Urutkan file berdasarkan bulan
csv_bulan_sorted = sorted(csv_bulan, key=lambda x: x[1])

# Gabungkan semua file sesuai urutan bulan
df_all = pd.concat([pd.read_csv(f[0]) for f in csv_bulan_sorted], ignore_index=True)

# Simpan hasil merge
output_path = 'data/data_tahunan/data_bmkg_2025_all.csv'
df_all.to_csv(output_path, index=False)
print(f"Data berhasil digabung dan disusun Jan-Des ke {output_path}")

Data berhasil digabung dan disusun Jan-Des ke data/data_tahunan/data_bmkg_2025_all.csv


In [85]:
# menggabungkan semua file tahunan dari 2019 hingga 2025

# Urutan tahun yang ingin digabung
tahun_order = ['2019', '2020', '2021', '2022', '2023', '2024', '2025']

# Ambil semua file tahunan yang sesuai urutan tahun
csv_files = [f"data/data_tahunan/data_bmkg_{tahun}_all.csv" for tahun in tahun_order if os.path.exists(f"data/data_tahunan/data_bmkg_{tahun}_all.csv")]

# Gabungkan semua file sesuai urutan tahun
df_all = pd.concat([pd.read_csv(f) for f in csv_files], ignore_index=True)

# Simpan hasil merge seluruh tahun
output_path = 'data/data_tahunan/data_bmkg_all_2019_2025.csv'
df_all.to_csv(output_path, index=False)
print(f"Data seluruh tahun berhasil digabung ke {output_path}")

Data seluruh tahun berhasil digabung ke data/data_tahunan/data_bmkg_all_2019_2025.csv
